In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import os,errno
import sys
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
import pandas as pd
import datetime as dt
from scipy.ndimage.measurements import label
from math import sin, cos, sqrt, atan2, radians
import geopy.distance
%matplotlib inline

dir2='/thorncroftlab_rit/ahenny/rain/'
dir1='/thorncroftlab_rit/ahenny/rain/US/ghcnd_all/'
dir='/thorncroftlab_rit/ahenny/rain/DISSERTATION_SCRIPTS_RESULTS/'

In [ ]:
#This script checks the day classifications for completeness.  It turns out, there are some days left out
#(when there is no precip anywhere...just a few days really).  Also, there are a few duplicated, when 
#there are equal amounts of precip from different categories (typically small amounts and 2 categories).
#This script corrects those duplicated by making the attribution to the more meaningful weather type 
#(i.e., pure extreme IVT instead of unspecified, or TC - AR combination instead of pure TC)

#NOTE: For EP days, this does not apply!  The above was for whole climatology.

yrs_neusa=np.arange(1979,2020,1)
season='summer'
for i in range(1,len(yrs_neusa)):
    year=yrs_neusa[i]
    if season=='winter':
        if year%4==0:
            feb_length=29
        else:
            feb_length=28
        base_date_1=dt.datetime(year-1,12,1,6)
        dates_list_1=[base_date_1+dt.timedelta(days=x) for x in range(31)]

        base_date_2=dt.datetime(year,1,1,6)
        dates_list_2=[base_date_2+dt.timedelta(days=x) for x in range(31+feb_length)]

        dates_list=dates_list_1+dates_list_2

    if season=='spring':
        base_date=dt.datetime(year,3,1,6)
        dates_list=[base_date+dt.timedelta(days=x) for x in range(31+30+31)]

    if season=='summer':
        base_date=dt.datetime(year,6,1,6)
        dates_list=[base_date+dt.timedelta(days=x) for x in range(30+31+31)]

    if season=='fall':
        base_date=dt.datetime(year,9,1,6)
        dates_list=[base_date+dt.timedelta(days=x) for x in range(30+31+30)]
        
    ds=xr.open_dataset(dir+'day_classifications_neusa_'+season+'_'+str(year)+'.nc')
    dates_t1_all=pd.DatetimeIndex(ds['dates_t1_all'].values)
    dates_t2_all=pd.DatetimeIndex(ds['dates_t2_all'].values)
    dates_t3_all=pd.DatetimeIndex(ds['dates_t3_all'].values)
    dates_t4_all=pd.DatetimeIndex(ds['dates_t4_all'].values)
    dates_t5_all=pd.DatetimeIndex(ds['dates_t5_all'].values)
    dates_t6_all=pd.DatetimeIndex(ds['dates_t6_all'].values)
    dates_t7_all=pd.DatetimeIndex(ds['dates_t7_all'].values)
    dates_t8_all=pd.DatetimeIndex(ds['dates_t8_all'].values)
    dates_t9_all=pd.DatetimeIndex(ds['dates_t9_all'].values)
    dates_t10_all=pd.DatetimeIndex(ds['dates_t10_all'].values)
    dates_t11_all=pd.DatetimeIndex(ds['dates_t11_all'].values)
    
    lists=[dates_t1_all,dates_t2_all,dates_t3_all,dates_t4_all,dates_t5_all,dates_t6_all,dates_t7_all,dates_t8_all,dates_t9_all,dates_t10_all,dates_t11_all]
    for j in range(len(lists)):
        if j==0:
            dates_all=lists[j]
        else:
            dates_all=dates_all.append(lists[j])
    dates_all=dates_all.sort_values()
    
    
    dates_ignored=[x for x in dates_list if x not in dates_all]
    dates_dup=[x for x in dates_list if len([y for y in dates_all if y==x])>1]
    for j in range(len(dates_all.unique())):
        if dates_all.unique()[j] in dates_dup:
            if dates_all[j] in dates_t11_all:
                t11_values=dates_t11_all.values
                removed=[x for x in t11_values if x!=pd.to_datetime(dates_all.unique()[j])]
                dates_t11_all=pd.DatetimeIndex(removed)
            elif dates_all[j] in dates_t8_all:
                t8_values=dates_t8_all.values
                removed=[x for x in t8_values if x!=pd.to_datetime(dates_all.unique()[j])]
                dates_t8_all=pd.DatetimeIndex(removed)
            elif dates_all[j] in dates_t1_all:
                t1_values=dates_t1_all.values
                removed=[x for x in t1_values if x!=pd.to_datetime(dates_all.unique()[j])]
                dates_t1_all=pd.DatetimeIndex(removed)
            elif dates_all[j] in dates_t4_all:
                t4_values=dates_t4_all.values
                removed=[x for x in t4_values if x!=pd.to_datetime(dates_all.unique()[j])]
                dates_t4_all=pd.DatetimeIndex(removed)
    
    lists=[dates_t1_all,dates_t2_all,dates_t3_all,dates_t4_all,dates_t5_all,dates_t6_all,dates_t7_all,dates_t8_all,dates_t9_all,dates_t10_all,dates_t11_all]
    for j in range(len(lists)):
        if j==0:
            dates_all=lists[j]
        else:
            dates_all=dates_all.append(lists[j])
    dates_all=dates_all.sort_values()
    
    print(len(dates_all))

    dates_ignored=[x for x in dates_list if x not in dates_all]
    #print(dates_ignored)
    dates_dup=[x for x in dates_list if len([y for y in dates_all if y==x])>1]
    #print(dates_dup)
    
if 1==1:
    
    dk=xr.Dataset()
    dk['dates_t1_all']=(('time1'),dates_t1_all)
    dk['dates_t2_all']=(('time2'),dates_t2_all)
    dk['dates_t3_all']=(('time3'),dates_t3_all)
    dk['dates_t4_all']=(('time4'),dates_t4_all)
    dk['dates_t5_all']=(('time5'),dates_t5_all)
    dk['dates_t6_all']=(('time6'),dates_t6_all)
    dk['dates_t7_all']=(('time7'),dates_t7_all)
    dk['dates_t8_all']=(('time8'),dates_t8_all)
    dk['dates_t9_all']=(('time9'),dates_t9_all)
    dk['dates_t10_all']=(('time10'),dates_t10_all)
    dk['dates_t11_all']=(('time11'),dates_t11_all)

    dk.coords['time1']=np.arange(len(dates_t1_all))
    dk.coords['time2']=np.arange(len(dates_t2_all))
    dk.coords['time3']=np.arange(len(dates_t3_all))
    dk.coords['time4']=np.arange(len(dates_t4_all))
    dk.coords['time5']=np.arange(len(dates_t5_all))
    dk.coords['time6']=np.arange(len(dates_t6_all))
    dk.coords['time7']=np.arange(len(dates_t7_all))
    dk.coords['time8']=np.arange(len(dates_t8_all))
    dk.coords['time9']=np.arange(len(dates_t9_all))
    dk.coords['time10']=np.arange(len(dates_t10_all))
    dk.coords['time11']=np.arange(len(dates_t11_all))

    try:
        os.remove(dir+'day_classifications_neusa_'+season+'_'+str(year)+'.nc')
    except OSError:
        pass
    dk.to_netcdf(dir+'day_classifications_neusa_'+season+'_'+str(year)+'.nc',mode='w',format='NETCDF4')

In [ ]:
#For EP day classifications (by which wtype produces most *extreme* precip)
season='fall'
thresh_choose=99
area_threshold_choose=80
season_choose=season

ds=xr.open_dataset(dir+'extreme_days_ghcnd_'+str(thresh_choose)+'_'+str(area_threshold_choose)+'_'+season_choose+'.nc')

lats=ds['lats'].values.tolist()
lons=ds['lons'].values.tolist()
lons=[x+360. for x in lons]
dates=ds['dates'].values
dates_unique=list(set(dates))
dates_unique=pd.DatetimeIndex(dates_unique).sort_values()
stations=ds['stations'].values.tolist()
obs=ds['obs'].values.tolist()
years=[x.year for x in dates_unique]
print(years)


if 1==1:
    ds=xr.open_dataset(dir+'day_classifications_neusa_'+season+'_ep_days'+'.nc')
    dates_t1_all=pd.DatetimeIndex(ds['dates_t1_all'].values)
    dates_t2_all=pd.DatetimeIndex(ds['dates_t2_all'].values)
    dates_t3_all=pd.DatetimeIndex(ds['dates_t3_all'].values)
    dates_t4_all=pd.DatetimeIndex(ds['dates_t4_all'].values)
    dates_t5_all=pd.DatetimeIndex(ds['dates_t5_all'].values)
    dates_t6_all=pd.DatetimeIndex(ds['dates_t6_all'].values)
    dates_t7_all=pd.DatetimeIndex(ds['dates_t7_all'].values)
    dates_t8_all=pd.DatetimeIndex(ds['dates_t8_all'].values)
    dates_t9_all=pd.DatetimeIndex(ds['dates_t9_all'].values)
    dates_t10_all=pd.DatetimeIndex(ds['dates_t10_all'].values)
    dates_t11_all=pd.DatetimeIndex(ds['dates_t11_all'].values)

    lists=[dates_t1_all,dates_t2_all,dates_t3_all,dates_t4_all,dates_t5_all,dates_t6_all,dates_t7_all,dates_t8_all,dates_t9_all,dates_t10_all,dates_t11_all]
    for j in range(len(lists)):
        if j==0:
            dates_all=lists[j]
        else:
            dates_all=dates_all.append(lists[j])
    dates_all=dates_all.sort_values()
    
    
    dates_ignored=[x for x in dates_unique if x not in dates_all]
    dates_dup=[x for x in dates_unique if len([y for y in dates_all if y==x])>1]
    
    print(dates_ignored)
    print(dates_dup)
    for j in range(len(dates_all.unique())):
        if dates_all.unique()[j] in dates_dup:
            if dates_all[j] in dates_t11_all:
                t11_values=dates_t11_all.values
                removed=[x for x in t11_values if x!=pd.to_datetime(dates_all.unique()[j])]
                dates_t11_all=pd.DatetimeIndex(removed)
            elif dates_all[j] in dates_t8_all:
                t8_values=dates_t8_all.values
                removed=[x for x in t8_values if x!=pd.to_datetime(dates_all.unique()[j])]
                dates_t8_all=pd.DatetimeIndex(removed)
            elif dates_all[j] in dates_t1_all:
                t1_values=dates_t1_all.values
                removed=[x for x in t1_values if x!=pd.to_datetime(dates_all.unique()[j])]
                dates_t1_all=pd.DatetimeIndex(removed)
            elif dates_all[j] in dates_t4_all:
                t4_values=dates_t4_all.values
                removed=[x for x in t4_values if x!=pd.to_datetime(dates_all.unique()[j])]
                dates_t4_all=pd.DatetimeIndex(removed)
    
    lists=[dates_t1_all,dates_t2_all,dates_t3_all,dates_t4_all,dates_t5_all,dates_t6_all,dates_t7_all,dates_t8_all,dates_t9_all,dates_t10_all,dates_t11_all]
    for j in range(len(lists)):
        if j==0:
            dates_all=lists[j]
        else:
            dates_all=dates_all.append(lists[j])
    dates_all=dates_all.sort_values()
    
    print(len(dates_all))

    dates_ignored=[x for x in dates_unique if x not in dates_all]
    print(dates_ignored)
    dates_dup=[x for x in dates_unique if len([y for y in dates_all if y==x])>1]
    print(dates_dup)

In [40]:
if 1==1:
    
    dk=xr.Dataset()
    dk['dates_t1_all']=(('time1'),dates_t1_all)
    dk['dates_t2_all']=(('time2'),dates_t2_all)
    dk['dates_t3_all']=(('time3'),dates_t3_all)
    dk['dates_t4_all']=(('time4'),dates_t4_all)
    dk['dates_t5_all']=(('time5'),dates_t5_all)
    dk['dates_t6_all']=(('time6'),dates_t6_all)
    dk['dates_t7_all']=(('time7'),dates_t7_all)
    dk['dates_t8_all']=(('time8'),dates_t8_all)
    dk['dates_t9_all']=(('time9'),dates_t9_all)
    dk['dates_t10_all']=(('time10'),dates_t10_all)
    dk['dates_t11_all']=(('time11'),dates_t11_all)

    dk.coords['time1']=np.arange(len(dates_t1_all))
    dk.coords['time2']=np.arange(len(dates_t2_all))
    dk.coords['time3']=np.arange(len(dates_t3_all))
    dk.coords['time4']=np.arange(len(dates_t4_all))
    dk.coords['time5']=np.arange(len(dates_t5_all))
    dk.coords['time6']=np.arange(len(dates_t6_all))
    dk.coords['time7']=np.arange(len(dates_t7_all))
    dk.coords['time8']=np.arange(len(dates_t8_all))
    dk.coords['time9']=np.arange(len(dates_t9_all))
    dk.coords['time10']=np.arange(len(dates_t10_all))
    dk.coords['time11']=np.arange(len(dates_t11_all))

    try:
        os.remove(dir+'day_classifications_neusa_'+season+'_ep_days'+'.nc')
    except OSError:
        pass
    dk.to_netcdf(dir+'day_classifications_neusa_'+season+'_ep_days'+'.nc',mode='w',format='NETCDF4')

DatetimeIndex(['2019-08-08 06:00:00'], dtype='datetime64[ns]', freq=None)
